In [1]:
import pandas as pd

In [54]:
def left_align(df: pd.DataFrame):
    """have booktitles align left"""
    # HT to commentators from https://stackoverflow.com/questions/17232013/how-to-set-the-pandas-dataframe-data-left-right-alignment/63869860#63869860
    
    # for some reason after aligning there is a trail of 0s after each decimal number -> remove those:
    try:
        df["ratio"] = df["ratio"].astype(str).str.replace("00000", "")
    except:
        pass
    try:
        df["rating"] = df["rating"].astype(str).str.replace("00000", "")
    except:
        pass
    
    left_aligned_df = df.style.set_properties(**{'text-align': 'left'})
    left_aligned_df = left_aligned_df.set_table_styles(
        [dict(selector='th', props=[('text-align', 'left')])]
    )
    return left_aligned_df

In [247]:
df = pd.read_csv("books.csv")
df2 = pd.read_csv("booksupdated.csv")
df = df.append(df2)
df["num_reviews"] = df["num_reviews"].astype(str).str.replace(",", "").astype(int)
df.to_csv("booksi.csv")

In [220]:
df = df.rename(columns = {'alt_title':'book'})

### Number of unique userIDs, books, and read books

In [221]:
df.shape[0]

136272

In [222]:
df["userID"].unique().shape[0]

276

In [223]:
df["book"].unique().shape[0]

59986

In [224]:
df[df["shelf"] == "read"]["book"].unique().shape[0]

26753

### Table for data screenshot

In [225]:
example = df.drop("Unnamed: 0", axis=1).tail()
example["userID"] = "exampleID"
example

,userID,shelf,title,book,author,avg_rating,num_reviews,rating,date_added,date_read
9744,exampleID,to-read,\n The Most Good You Can Do: How Effectiv...,The Most Good You Can Do: How Effective Altrui...,"Singer, Peter",3.85,2489,NaN,2021-03-13 00:00:00,not set
9745,exampleID,to-read,\n Doing Good Better: How Effective Altru...,Doing Good Better: How Effective Altruism Can ...,"MacAskill, William",4.25,4214,NaN,2021-03-13 00:00:00,not set
9746,exampleID,to-read,\n The Precipice: Existential Risk and th...,The Precipice: Existential Risk and the Future...,"Ord, Toby",4.23,1260,NaN,2021-03-13 00:00:00,not set
9747,exampleID,to-read,\n Moral Uncertainty\n,Moral Uncertainty,"MacAskill, William",4.29,28,NaN,2021-03-13 00:00:00,not set
9748,exampleID,to-read,"\n Guns, Germs, and Steel: The Fates of H...","Guns, Germs, and Steel: The Fates of Human Soc...","Diamond, Jared",4.03,323031,NaN,2021-03-13 00:00:00,not set


## Most read books

In [226]:
df_most_read = df[df["shelf"] == "read"].groupby(["book"]).count().sort_values(by=["userID"], ascending=False).drop("Unnamed: 0", axis=1)
to_drop = ['shelf', 'title', 'author', 'avg_rating', 'num_reviews',
       'rating', 'date_added', 'date_read']
left_align(df_most_read.head(40).drop(to_drop, axis=1).rename(columns = {'userID':'n'}))

,n
book,
1984,130
"Thinking, Fast and Slow",124
Animal Farm,105
Sapiens: A Brief History of Humankind,102
"Superintelligence: Paths, Dangers, Strategies",102
Harry Potter and the Sorcerer's Stone (Harry Potter #1),92
Doing Good Better: How Effective Altruism Can Help You Make a Difference,91
Brave New World,89
"The Hitchhiker's Guide to the Galaxy (Hitchhiker's Guide to the Galaxy, #1)",87


## Most planned to read

In [227]:
to_drop = ['shelf', 'title', 'author', 'avg_rating', 'num_reviews',
       'rating', 'date_added', 'date_read', "Unnamed: 0"]
left_align(df[df["shelf"] == "to-read"].groupby(["book"]).count().sort_values(by=["userID"], ascending=False).head(40).drop(to_drop, axis=1).rename(columns = {'userID':'n'}))

,n
book,
The Better Angels of Our Nature: Why Violence Has Declined,63
"Gödel, Escher, Bach: an Eternal Golden Braid",63
Reasons and Persons,62
Superforecasting: The Art and Science of Prediction,61
Algorithms to Live By: The Computer Science of Human Decisions,54
Man's Search for Meaning,53
"Why Nations Fail: The Origins of Power, Prosperity, and Poverty",50
"Guns, Germs, and Steel: The Fates of Human Societies",50
Factfulness: Ten Reasons We're Wrong About the World – and Why Things Are Better Than You Think,50


## Planned-to-read/Have-read Ratio, or: Maybe more people want to read those books?

In [228]:
df_merged = df[df["shelf"] == "to-read"].groupby(["book"]).count().merge(
    df[df["shelf"] == "read"].groupby(["book"]).count(), on=["book"])
df_ratios = pd.DataFrame(df_merged["userID_x"]/df_merged["userID_y"], columns=["ratio"])#.sort_values(by="ratio", ascending=False).head(30).round(2)

In [229]:
drop_them = list(df_most_read.columns)
drop_them.remove("userID")
drop_them

['shelf',
 'title',
 'author',
 'avg_rating',
 'num_reviews',
 'rating',
 'date_added',
 'date_read']

In [230]:
left_align(df_ratios.merge(df_most_read, on=["book"]).sort_values(by="ratio", ascending=False).head(50).drop(drop_them, axis=1).rename(columns = {'userID':'n'}).astype(int))

,ratio,n
book,,
The Scout Mindset: Why Some People See Things Clearly and Others Don't,24,1
Probability Theory,22,1
The Evolution of Cooperation,21,1
The Great Transformation: The Political and Economic Origins of Our Time,20,1
The Art of Doing Science and Engineering: Learning to Learn,19,1
Poor Charlie's Almanack: The Wit and Wisdom of Charles T. Munger,18,1
The Human Condition,18,1
Collapse of Complex Societies,17,1
The True Believer: Thoughts on the Nature of Mass Movements,16,1


In [231]:
left_align(df_ratios.merge(df_most_read[df_most_read["userID"] > 1], on=["book"]).sort_values(by="ratio", ascending=False).head(40).drop(drop_them, axis=1).round(2).rename(columns = {'userID':'n'}))

,ratio,n
book,,
Rationality and the Reflective Mind,13.0,2
The Rise and Fall of American Growth: The U.S. Standard of Living Since the Civil War,11.0,2
The Road to Reality: A Complete Guide to the Laws of the Universe,10.5,2
The Logic of Scientific Discovery,9.5,2
Complexity: A Guided Tour,9.5,2
Structures: Or Why Things Don't Fall Down,9.5,2
The Denial of Death,9.0,2
The Value of Everything: Making and Taking in the Global Economy,8.5,2
Maps of Meaning: The Architecture of Belief,8.5,2


In [232]:
left_align(pd.DataFrame(df_merged["userID_x"]/df_merged["userID_y"], columns=["ratio"]).sort_values(by="ratio", ascending=True).head(20).round(2))

,ratio
book,
"Harry Potter and the Prisoner of Azkaban (Harry Potter, #3)",0.01
"Harry Potter and the Chamber of Secrets (Harry Potter, #2)",0.01
"Harry Potter and the Goblet of Fire (Harry Potter, #4)",0.01
"Harry Potter and the Half-Blood Prince (Harry Potter, #6)",0.01
"Harry Potter and the Deathly Hallows (Harry Potter, #7)",0.01
"Harry Potter and the Order of the Phoenix (Harry Potter, #5)",0.01
"The Fellowship of the Ring (The Lord of the Rings, #1)",0.02
Harry Potter and the Sorcerer's Stone (Harry Potter #1),0.02
"The Return of the King (The Lord of the Rings, #3)",0.03


### Most wanted books that have never been read

In [233]:
to_drop = ['shelf', 'title', 'author', 'avg_rating', 'num_reviews',
       'rating', 'date_added', 'date_read', "Unnamed: 0"]

In [234]:
common = df[df["shelf"] == "to-read"].merge(df[df["shelf"] == "read"],on=['book'])
left_align(df[df["shelf"] == "to-read"][(~df[df["shelf"] == "to-read"].book.isin(common.book))]\
        .groupby(["book"]).count().sort_values(by=["userID"], ascending=False).head(40).drop(to_drop, axis=1).rename(columns = {'userID':'n'}))

,n
book,
Energy and Civilization: A History,25
Think Again: The Power of Knowing What You Don't Know,17
The Oxford Handbook of Thinking and Reasoning,17
The Innovator's Dilemma: The Revolutionary Book that Will Change the Way You Do Business,16
Simulacra and Simulation,15
Why Greatness Cannot Be Planned: The Myth of the Objective,15
Growth: From Microorganisms to Megacities,14
Reinventing Organizations: A Guide to Creating Organizations Inspired by the Next Stage of Human Consciousness,14
Trick Mirror: Reflections on Self-Delusion,14


## Highest rated

In [235]:
df["rating"].unique()

array(['liked it', 'really liked it', 'it was ok', 'it was amazing',
       'did not like it', nan], dtype=object)

In [236]:
df = df.replace({"liked it": 3, "really liked it": 4, "it was ok": 2, "it was amazing": 5, "did not like it": 1})

In [237]:
relevant_titles = list(df_most_read[df_most_read["userID"] > 10].index)

In [238]:
to_drop = ['avg_rating_x', 'num_reviews_x', 'userID_x', 'shelf_x',
       'title_x', 'author_x', 'avg_rating_y', 'num_reviews_y', 'rating_y',
       'date_added_x', 'date_read_x', 'shelf_y', 'title_y',
       'author_y', 'rating', 'date_added_y', 'date_read_y', "Unnamed: 0", "avg_rating", "num_reviews"]

In [239]:
left_align(df.groupby(["book"]).mean().merge(df_most_read, on=["book"]).merge(df_most_read, on=["book"]).loc[relevant_titles].sort_values(by=["rating_x"], ascending=False).head(40).drop(to_drop, axis=1).round(2).rename(columns = {'userID_y':'n', 'rating_x': 'rating'}))

,rating,n
book,,
"Famine, Affluence, and Morality",4.86,21
Doing Good Better: How Effective Altruism Can Help You Make a Difference,4.82,91
Mother of Learning,4.82,11
The Precipice: Existential Risk and the Future of Humanity,4.79,78
All the Light We Cannot See,4.78,11
Reasons and Persons,4.75,20
Harry Potter and the Methods of Rationality,4.75,65
"The Ultimate Hitchhiker's Guide to the Galaxy (Hitchhiker's Guide to the Galaxy, #1-5)",4.74,24
Infinite Jest,4.71,15


### Lowest rated

In [240]:
left_align(df.groupby(["book"]).mean().merge(df_most_read, on=["book"]).merge(df_most_read, on=["book"]).loc[relevant_titles].sort_values(by=["rating_x"], ascending=True).head(30).drop(to_drop, axis=1).round(2).rename(columns = {'userID_y':'n', 'rating_x': 'rating'}))

,rating,n
book,,
"New Moon (Twilight, #2)",1.83,11
"The Secret (The Secret, #1)",1.85,13
"Twilight (Twilight, #1)",1.86,26
The Sun Also Rises (Fiesta),2.56,11
"Brisingr (The Inheritance Cycle, #3)",2.67,11
"Eat, Pray, Love",2.67,11
Anthem,2.9,14
"Eldest (The Inheritance Cycle, #2)",2.91,13
The Communist Manifesto,2.91,28


## Fringe books read by EAs

In [241]:
df_most_read_only = df_most_read[df_most_read["userID"] > 10]

In [242]:
to_drop = ['shelf', 'title', 'author', 'avg_rating_y',
       'num_reviews_y', 'rating_y', 'date_added', 'date_read', "avg_rating_x", "rating_x"]

In [245]:
left_align(df.groupby(["book"]).mean().round(0).drop("Unnamed: 0", axis=1).merge(df_most_read_only, on="book")\
    .sort_values(by="num_reviews_x", ascending=True).head(30).drop(to_drop, axis=1).astype(int)\
    .rename(columns = {'num_reviews_x':'G. reviews', 'userID': 'EA reads'}))

,G. reviews,EA reads
book,,
The Replacing Guilt Series,45,12
Effective Altruism Handbook,108,18
The Less Wrong Sequences,203,13
"Crystal Mentality (Crystal Trilogy, #2)",215,13
Global Catastrophic Risks,217,12
"The AI Does Not Hate You: Superintelligence, Rationality and the Race to Save the World",231,21
What Intelligence Tests Miss: The Psychology of Rational Thought,236,13
…And I Show You How Deep the Rabbit Hole Goes,246,12
The Sword of Good,312,11
